연	월	일	시	건물번호	기온(C)	강수량(mm)	풍속(m/s)	습도(%)
 - XGBRegressor: 0.9655916940702893
 - Dacon 점수: 17.44154

'건물번호', '일시', '기온(C)', '강수량(mm)', '풍속(m/s)', '습도(%)', '일조(hr)', '일사(MJ/m2)', '연', '월', '일', '시', '연면적(m2)',
       '냉방면적(m2)', '태양광용량(kW)', 'ESS저장용량(kWh)', 'PCS용량(kW)', '건물기타', '공공',
       '대학교', '데이터센터', '백화점및아울렛', '병원', '상용', '아파트', '연구소', '지식산업센터', '할인마트',
       '호텔및리조트''

In [58]:
import pandas as pd

In [59]:
building_info = pd.read_csv('./data/building_info.csv')
test = pd.read_csv('./data/test.csv')
train = pd.read_csv('./data/train.csv')
sample = pd.read_csv('./data/sample_submission.csv')
train.head(1)

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh)
0,1_20220601 00,1,20220601 00,18.6,NaN,0.9,42.0,NaN,NaN,1085.28


In [60]:
test.head(1)

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%)
0,1_20220825 00,1,20220825 00,23.5,0.0,2.2,72


In [61]:
building_info.head(1)

,건물번호,건물유형,연면적(m2),냉방면적(m2),태양광용량(kW),ESS저장용량(kWh),PCS용량(kW)
0,1,건물기타,110634.0,39570.0,-,-,-


In [62]:
from sklearn.preprocessing import OneHotEncoder

building_info.replace('-', 0, inplace=True)
onehotencoder = OneHotEncoder(sparse_output=False)
building_type = onehotencoder.fit_transform(building_info[['건물유형']])
print(onehotencoder.categories_[0])

temp = pd.DataFrame(building_type)
temp.columns = onehotencoder.categories_[0]
temp = temp.astype('int32')

building = pd.concat([building_info, temp], axis=1).head(3)
del building['건물유형']
building.head(3)

['건물기타' '공공' '대학교' '데이터센터' '백화점및아울렛' '병원' '상용' '아파트' '연구소' '지식산업센터' '할인마트'
 '호텔및리조트']


,건물번호,연면적(m2),냉방면적(m2),태양광용량(kW),ESS저장용량(kWh),PCS용량(kW),건물기타,공공,대학교,데이터센터,백화점및아울렛,병원,상용,아파트,연구소,지식산업센터,할인마트,호텔및리조트
0,1,110634.00,39570.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,2,122233.47,99000.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,3,171243.00,113950.0,40,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [63]:
def data_preprocessing(data):  
    data.fillna(0, inplace=True)
    data['연'] = data['num_date_time'].apply(lambda x: int(x.split('_')[1][:4]))
    data['월'] = data['num_date_time'].apply(lambda x: int(x.split('_')[1][4:6]))
    data['일'] = data['num_date_time'].apply(lambda x: int(x.split('_')[1][6:8]))
    data['시'] = data['num_date_time'].apply(lambda x: int(x.split(' ')[1]))
    del data['일시']
    del data['num_date_time']
    
    data = pd.merge(data, building, on='건물번호')
    del data['건물번호']
    display(data.head(3))
    return data
    
train = data_preprocessing(train)

,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh),연,월,일,...,대학교,데이터센터,백화점및아울렛,병원,상용,아파트,연구소,지식산업센터,할인마트,호텔및리조트
0,18.6,0.0,0.9,42.0,0.0,0.0,1085.28,2022,6,1,...,0,0,0,0,0,0,0,0,0,0
1,18.0,0.0,1.1,45.0,0.0,0.0,1047.36,2022,6,1,...,0,0,0,0,0,0,0,0,0,0
2,17.7,0.0,1.5,45.0,0.0,0.0,974.88,2022,6,1,...,0,0,0,0,0,0,0,0,0,0


In [64]:
print(train.columns)

Index(['기온(C)', '강수량(mm)', '풍속(m/s)', '습도(%)', '일조(hr)', '일사(MJ/m2)',
       '전력소비량(kWh)', '연', '월', '일', '시', '연면적(m2)', '냉방면적(m2)', '태양광용량(kW)',
       'ESS저장용량(kWh)', 'PCS용량(kW)', '건물기타', '공공', '대학교', '데이터센터', '백화점및아울렛',
       '병원', '상용', '아파트', '연구소', '지식산업센터', '할인마트', '호텔및리조트'],
      dtype='object')


In [ ]:
columns = '연 월 일 시 건물번호 기온(C) 강수량(mm) 풍속(m/s) 습도(%)'.split()
columns = '건물번호', '일시', '기온(C)', '강수량(mm)', '풍속(m/s)', '습도(%)',
       '일조(hr)', '일사(MJ/m2)', '전력소비량(kWh)', '연', '월', '일', '시', '연면적(m2)',
       '냉방면적(m2)', '태양광용량(kW)', 'ESS저장용량(kWh)', 'PCS용량(kW)', '건물기타', '공공',
       '대학교', '데이터센터', '백화점및아울렛', '병원', '상용', '아파트', '연구소', '지식산업센터', '할인마트',
       '호텔및리조트''
data = train[columns]
y = train['전력소비량(kWh)']
data.head(1)

,연,월,일,시,건물번호,기온(C),강수량(mm),풍속(m/s),습도(%)
0,2022,6,1,0,1,18.6,0.0,0.9,42.0


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.25, random_state=987)


In [ ]:
from xgboost import XGBRegressor
from sklearn.metrics import r2_score

XR = XGBRegressor()
XR.fit(X_train, y_train)
y_pred = XR.predict(X_test)
r2_score(y_test, y_pred)


0.9655916940702893

In [ ]:
test_data = data_preprocessing(test)
test_data = test_data[columns]
test_data.head(3)

,건물번호,기온(C),강수량(mm),풍속(m/s),습도(%),연,월,일,시
0,1,23.5,0.0,2.2,72,2022,8,25,0
1,1,23.0,0.0,0.9,72,2022,8,25,1
2,1,22.7,0.0,1.5,75,2022,8,25,2


,연,월,일,시,건물번호,기온(C),강수량(mm),풍속(m/s),습도(%)
0,2022,8,25,0,1,23.5,0.0,2.2,72
1,2022,8,25,1,1,23.0,0.0,0.9,72
2,2022,8,25,2,1,22.7,0.0,1.5,75


In [ ]:
answer = XR.predict(test_data)
answer

array([1984.8647 , 1963.7201 , 1884.4098 , ...,  715.0567 ,  618.64276,
        496.988  ], dtype=float32)

In [ ]:
submission = pd.DataFrame()
submission['num_date_time'] = sample['num_date_time']
submission['answer'] = answer
submission.head()

,num_date_time,answer
0,1_20220825 00,1984.864746
1,1_20220825 01,1963.720093
2,1_20220825 02,1884.409790
3,1_20220825 03,1938.069580
4,1_20220825 04,1925.296143


In [ ]:
import datetime

now = datetime.datetime.now().strftime("%Y-%m-%d %H시%M분%S초")
# submission.to_csv(f'{now}.csv', index=False)